# Managing Outputs with Output Parsers

## 1-1. PydanticOutputParser

This class instructs the model to generate its output in a JSON format and then extract the information from the response. You will be able to treat the parser’s output as a list, meaning it will be possible to index through the results without worrying about formatting.

💡
It is important to note that not all models have the same capability in terms of generating JSON outputs. So, it would be best to use a more powerful model (like OpenAI’s DaVinci instead of Curie) to get the most satisfactory result.

This class uses the Pydantic library, which helps define and validate data structures in Python. It enables us to characterize the expected output with a name, type, and description. We need a variable that can store multiple suggestions in the thesaurus example. It can be easily done by defining a class that inherits from the Pydantic’s BaseModel class. Remember to install the required packages with the following command: pip install langchain==0.1.4 deeplake openai==1.10.0 tiktoken.

In [1]:
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, validator
from typing import List

# Define your desired data structure.
class Suggestions(BaseModel):
    words: List[str] = Field(description="list of substitue words based on context")

    # Throw error in case of receiving a numbered-list from API
    @validator('words')
    def not_start_with_number(cls, field):
        for item in field:
            if item[0].isnumeric():
                raise ValueError("The word can not start with numbers!")
        return field

parser = PydanticOutputParser(pydantic_object=Suggestions)

We always import and follow the necessary libraries by creating the Suggestions schema class. There are two essential parts to this class:

1. Expected Outputs: Each output is defined by declaring a variable with desired type, like a list of strings (: List[str]) in the sample code, or it could be a single string (: str) if you are expecting just one word/sentence as the response. Also, It is required to write a simple explanation using the Field function’s description attribute to help the model during inference. (We will see an example of having multiple outputs later in the lesson)
2. Validators: It is possible to declare functions to validate the formatting. We ensure that the first character is not a number in the sample code. The function’s name is unimportant, but the @validator decorator must receive the same name as the variable you want to approve. (like @validator(’words’)) It is worth noting that the field variable inside the validator function will be a list if you specify it as one.


We will pass the created class to the PydanticOutputParser wrapper to make it a LangChain parser object. The next step is to prepare the prompt.

In [2]:
from langchain.prompts import PromptTemplate

template = """
Offer a list of suggestions to substitue the specified target_word based the presented context.
{format_instructions}
target_word={target_word}
context={context}
"""

prompt = PromptTemplate(
    template=template,
    input_variables=["target_word", "context"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

model_input = prompt.format_prompt(
			target_word="behaviour",
			context="The behaviour of the students in the classroom was disruptive and made it difficult for the teacher to conduct the lesson."
)

As discussed in previous lessons, the template variable is a string that can have named index placeholders using the following {variable_name} format. The template outlines our expectations for the model, including the expected formatting from the parser and the inputs. The PromptTemplate receives the template string with the details of each placeholder’s type. They could either be 1) input_variables whose value is initialized later on using the .format_prompt() function, or 2) partial_variables to be initialized instantly. 

The prompt can send the query to models like GPT using LangChain’s OpenAI wrapper. (Remember to set the OPENAI_API_KEY environment variables with your API key from OpenAI) We are using the Davinci model, one of the more powerful options to get the best results, and set the temperature value to 0, making the results reproducible.

💡The temperature value could be anything between 0 to 1, where a higher number means the model is more creative. Using larger value in production is a good practice if you deal with tasks requiring creative output.

In [7]:
from langchain.llms import OpenAI

# Before executing the following code, make sure to have
# your OpenAI key saved in the “OPENAI_API_KEY” environment variable.
model = OpenAI(model_name='gpt-3.5-turbo-instruct', temperature=0.0)

output = model(model_input.to_string())

parser.parse(output)

Suggestions(words=['conduct', 'manage', 'handle', 'oversee', 'supervise'])

The sample code.

Suggestions(words=['conduct', 'manner', 'action', 'demeanor', 'attitude', 'activity'])

The output.

The parser object’s parse() function will convert the model’s string response to the format we specified. There is a list of words that you can index through and use in your applications.


### Multiple Outputs Example

Here is a sample code for Pydantic class to process multiple outputs. It requests the model to suggest a list of words and present the reasoning behind each proposition.

Replace the template variable and Suggestion class with the following codes to run this example. The template changes will ask the model to present its reasoning, and the suggestion class declares a new output named reasons. Also, the validator function manipulates the output to ensure every reasoning ends with a dot. Another use case of the validator function could be output manipulation.

In [8]:
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, validator
from typing import List

# Define your desired data structure. (Multi)
class Suggestions(BaseModel):
    words: List[str] = Field(description="list of substitue words based on context")
    reasons: List[str] = Field(description="the reasoning of why this word fits the context")
    
    @validator('words')
    def not_start_with_number(cls, field):
      for item in field:
        if item[0].isnumeric():
          raise ValueError("The word can not start with numbers!")
      return field
    
    @validator('reasons')
    def end_with_dot(cls, field):
      for idx, item in enumerate( field ):
        if item[-1] != ".":
          field[idx] += "."
      return field

parser = PydanticOutputParser(pydantic_object=Suggestions)

In [9]:
from langchain.prompts import PromptTemplate

template = """
Offer a list of suggestions to substitute the specified target_word based on the presented context and the reasoning for each word.
{format_instructions}
target_word={target_word}
context={context}
"""

prompt = PromptTemplate(
    template=template,
    input_variables=["target_word", "context"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

model_input = prompt.format_prompt(
			target_word="behaviour",
			context="The behaviour of the students in the classroom was disruptive and made it difficult for the teacher to conduct the lesson."
)

In [10]:
from langchain.llms import OpenAI

# Before executing the following code, make sure to have
# your OpenAI key saved in the “OPENAI_API_KEY” environment variable.
model = OpenAI(model_name='gpt-3.5-turbo-instruct', temperature=0.0)

output = model(model_input.to_string())

parser.parse(output)

Suggestions(words=['conduct', 'manage', 'handle', 'oversee'], reasons=["These words all imply a sense of control and authority, which is lacking in the original context. They also suggest a more active role in guiding the students' actions.", "These words all suggest a more professional and organized approach to managing the students' actions, which contrasts with the disruptive behaviour described in the original context.", "These words all imply a sense of responsibility and authority, which is lacking in the original context. They also suggest a more active role in guiding the students' actions.", "These words all suggest a more authoritative and organized approach to overseeing the students' actions, which contrasts with the disruptive behaviour described in the original context."])

The multiple outputs sample code.

Suggestions(words=['conduct', 'manner', 'demeanor', 'comportment'], reasons=['refers to the way someone acts in a particular situation.', 'refers to the way someone behaves in a particular situation.', 'refers to the way someone behaves in a particular situation.', 'refers to the way someone behaves in a particular situation.'])

The output.

## 1-2. CommaSeparatedOutputParser

It is evident from the name of this class that it manages comma-separated outputs. It handles one specific case: anytime you want to receive a list of outputs from the model. Let’s start by importing the necessary module.

In [18]:
from langchain.output_parsers import CommaSeparatedListOutputParser

parser = CommaSeparatedListOutputParser()

The parser does not require a setting up step. Therefore it is less flexible. We can create the object by calling the class. The rest of the process for writing the prompt, initializing the model, and parsing the output is as follows.

In [22]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate

# Prepare the Prompt
template = """
Offer a list of suggestions to substitute the word '{target_word}' based the presented the following text: {context}.
{format_instructions}
"""

prompt = PromptTemplate(
    template=template,
    input_variables=["target_word", "context"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

model_input = prompt.format(
  target_word="behaviour",
  context="The behaviour of the students in the classroom was disruptive and made it difficult for the teacher to conduct the lesson."
)

# Loading OpenAI API
model = OpenAI(model_name='gpt-3.5-turbo-instruct', temperature=0.0)



# Send the Request
output = model(model_input)
parser.parse(output)

['1. Conduct\n2. Manner\n3. Demeanor\n4. Attitude\n5. Conducting\n6. Actions\n7. Conducted\n8. Conductivity\n9. Deportment\n10. Etiquette']

The sample code.

['Conduct',
 'Actions',
 'Demeanor',
 'Mannerisms',
 'Attitude',
 'Performance',
 'Reactions',
 'Interactions',
 'Habits',
 'Repertoire',
 'Disposition',
 'Bearing',
 'Posture',
 'Deportment',
 'Comportment']

The output.

Although most of the sample code has been explained in the previous subsection, two parts might need attention. Firstly, we tried a new format for the prompt’s template to show different ways to write a prompt. Secondly, the use of .format() instead of .format_prompt() to generate the model’s input. The main difference compared to the previous subsection’s code is that we no longer need to call the .to_string() object since the prompt is already in string type.

As you can see, the final output is a list of words that has some overlaps with the PydanticOutputParser approach with more variety. However, requesting additional reasoning information using the CommaSeparatedOutputParser class is impossible.

Testing the langchain doc example:

In [1]:
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

output_parser = CommaSeparatedListOutputParser()

format_instructions = output_parser.get_format_instructions()
prompt = PromptTemplate(
    template="List five {subject}.\n{format_instructions}",
    input_variables=["subject"],
    partial_variables={"format_instructions": format_instructions},
)

model = ChatOpenAI(temperature=0)

chain = prompt | model | output_parser

In [2]:
chain.invoke({"subject": "ice cream flavors"})

['Vanilla',
 'Chocolate',
 'Strawberry',
 'Mint Chocolate Chip',
 'Cookies and Cream']

Testing the course code with chat model

In [10]:
from langchain.output_parsers import CommaSeparatedListOutputParser

parser = CommaSeparatedListOutputParser()

In [18]:
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# Prepare the Prompt
template = """
Offer a list of suggestions to substitute the word '{target_word}' based the presented the following text: {context}.
{format_instructions}
"""

prompt = PromptTemplate(
    template=template,
    input_variables=["target_word", "context"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

model_input = prompt.format(
  target_word="behaviour",
  context="The behaviour of the students in the classroom was disruptive and made it difficult for the teacher to conduct the lesson."
)

# Loading OpenAI API
#model = OpenAI(model_name='gpt-3.5-turbo-instruct', temperature=0.0)
model = ChatOpenAI(temperature=0.0)


# Send the Request
output = model(model_input)
parser.parse(output)

AttributeError: 'str' object has no attribute 'content'

Still not the expected output format with comma seperators

Testing the course code with different calling methods for the parser:


In [7]:
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI


new_parser = CommaSeparatedListOutputParser()

template2 = """
Offer a list of suggestions to substitute the word '{target_word2}' based the presented the following text: {context2}.
{formatted_instructions}
"""

formatted_instructions = new_parser.get_format_instructions()
prompt = PromptTemplate(
    template=template2,
    input_variables=["target_word2", "context2"],
    partial_variables={"formatted_instructions": formatted_instructions},
)

model = ChatOpenAI(temperature=0)

chain = prompt | model | output_parser

In [9]:
chain.invoke({"target_word2":"behaviour", "context2":"The behaviour of the students in the classroom was disruptive and made it difficult for the teacher to conduct the lesson."})

['conduct', 'actions', 'demeanor', 'conduct', 'mannerisms']